In [23]:
import pandas as pd
import numpy as np
import pickle
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings('ignore')


### 1. Load Data

In [24]:
ratings = pd.read_csv('data/processed/processed_ratings.csv')
movies = pd.read_csv('data/processed/processed_movies.csv')

print(f"✓ Loaded {len(ratings):,} ratings")
print(f"✓ Loaded {len(movies):,} movies")


✓ Loaded 100,000 ratings
✓ Loaded 1,682 movies


### 2. User Item Matrix

In [25]:
user_item_matrix = ratings.pivot_table(
    index='user_id',
    columns='item_id',
    values='rating',
    fill_value=0
)

print(f"✓ Matrix shape: {user_item_matrix.shape}")
print(f"✓ Sparsity: {(1 - (ratings.shape[0] / (user_item_matrix.shape[0] * user_item_matrix.shape[1])))*100:.2f}%")

✓ Matrix shape: (943, 1682)
✓ Sparsity: 93.70%


### 3. Train-Test Split

In [26]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings, test_size=0.2, stratify=ratings['user_id'], random_state=42)
print(f"✓ Training set: {len(train_data):,} ratings")
print(f"✓ Test set: {len(test_data):,} ratings")

# Create train matrix
train_matrix = train_data.pivot_table(
    index='user_id',
    columns='item_id',
    values='rating',
    fill_value=0
)

✓ Training set: 80,000 ratings
✓ Test set: 20,000 ratings


### 4. Evaluation 

In [27]:
import numpy as np
import pandas as pd
import math

def calculate_metrics(predictions, test_data, movies_df, k=10):
    """
    Calculate comprehensive evaluation metrics: Precision, Recall, NDCG, and Coverage.
    
    Args:
        predictions (dict): Dictionary mapping user_id to a list of recommended item_ids.
        test_data (pd.DataFrame): The test dataset containing actual user ratings.
        movies_df (pd.DataFrame): The movies dataset (used for coverage calculation).
        k (int): Number of top recommendations to evaluate.
        
    Returns:
        dict: A dictionary containing the calculated metrics.
    """
    precisions = []
    recalls = []
    ndcgs = []
    
    # OPTIMIZATION: Pre-group test data by user to avoid slow filtering inside the loop
    # This converts the DataFrame into a dictionary {user_id: set(item_ids)}
    actual_user_items = test_data.groupby('user_id')['item_id'].apply(set).to_dict()
    
    for user_id, actual_items in actual_user_items.items():
        # We only evaluate users who actually received predictions
        if user_id in predictions and len(actual_items) > 0:
            
            # Get top-k predictions
            # Note: We use a list here to preserve order for NDCG calculation
            pred_list = predictions[user_id][:k]
            
            if not pred_list:
                continue
                
            # --- 1. Identify Hits ---
            # Create a binary list (1 if item is relevant, 0 if not)
            hits = [1 if item in actual_items else 0 for item in pred_list]
            num_hits = sum(hits)
            
            # --- 2. Precision@K ---
            # (Relevant Items Found) / K
            precisions.append(num_hits / k)
            
            # --- 3. Recall@K ---
            # (Relevant Items Found) / (Total Relevant Items in Test Set)
            recalls.append(num_hits / len(actual_items))
            
            # --- 4. NDCG@K (Ranking Quality) ---
            # Calculate DCG: sum(rel_i / log2(rank_i + 1))
            # We use idx+2 because enumerate starts at 0, and log formula uses rank (1-based) + 1
            dcg = sum([rel / np.log2(idx + 2) for idx, rel in enumerate(hits)])
            
            # Calculate IDCG: Ideal DCG (if all hits were at the very top)
            possible_hits = min(len(actual_items), k)
            idcg = sum([1.0 / np.log2(idx + 2) for idx in range(possible_hits)])
            
            if idcg > 0:
                ndcgs.append(dcg / idcg)
            else:
                ndcgs.append(0)

    # --- 5. Coverage ---
    # Count unique items recommended across all users
    all_recommended_items = set()
    for recs in predictions.values():
        all_recommended_items.update(recs[:k])
    
    total_movies = movies_df['movie_id'].nunique()
    coverage = len(all_recommended_items) / total_movies if total_movies > 0 else 0

    return {
        f'Precision@{k}': np.mean(precisions) if precisions else 0.0,
        f'Recall@{k}': np.mean(recalls) if recalls else 0.0,
        f'NDCG@{k}': np.mean(ndcgs) if ndcgs else 0.0,
        'Coverage': coverage,
        'Avg_Recs_Per_User': np.mean([len(v) for v in predictions.values()])
    }

### 5. User Based Collaborative Filtering

In [28]:
start_time = time.time()

# Calculate user similarity
user_similarity = cosine_similarity(train_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=train_matrix.index,
    columns=train_matrix.index
)

def user_based_recommendations(user_id, n=10, n_neighbors=20):
    """Generate recommendations using user-based CF"""
    if user_id not in user_similarity_df.index:
        return []
    
    # Get similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:n_neighbors+1]
    
    # Get items the user hasn't rated
    user_items = set(train_data[train_data['user_id'] == user_id]['item_id'].values)
    
    # Score items based on similar users' ratings
    item_scores = {}
    for sim_user, similarity in similar_users.items():
        sim_user_items = train_data[train_data['user_id'] == sim_user]
        for _, row in sim_user_items.iterrows():
            if row['item_id'] not in user_items:
                if row['item_id'] not in item_scores:
                    item_scores[row['item_id']] = 0
                item_scores[row['item_id']] += similarity * row['rating']
    
    # Sort and return top N
    recommendations = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:n]
    return [item_id for item_id, score in recommendations]

# Generate predictions for test users
user_based_predictions = {}
test_users = test_data['user_id'].unique()  
for user_id in test_users:
    user_based_predictions[user_id] = user_based_recommendations(user_id)

training_time_ub = time.time() - start_time
metrics_ub = calculate_metrics(user_based_predictions, test_data, movies)

print(f"✓ Training time: {training_time_ub:.2f} seconds")
print(f"✓ Precision@10:  {metrics_ub['Precision@10']:.4f}")
print(f"✓ Recall@10:     {metrics_ub['Recall@10']:.4f}")
print(f"✓ NDCG@10:       {metrics_ub['NDCG@10']:.4f}")
print(f"✓ Coverage:      {metrics_ub['Coverage']:.4f}")

✓ Training time: 72.28 seconds
✓ Precision@10:  0.3203
✓ Recall@10:     0.2095
✓ NDCG@10:       0.3837
✓ Coverage:      0.2004


### 6. Item Based Collaborative Filtering

In [29]:
start_time = time.time()

# Calculate item similarity
item_similarity = cosine_similarity(train_matrix.T)
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=train_matrix.columns,
    columns=train_matrix.columns
)

def item_based_recommendations(user_id, n=10):
    """Generate recommendations using item-based CF"""
    # Get items the user has rated
    user_items = train_data[train_data['user_id'] == user_id]
    
    if len(user_items) == 0:
        return []
    
    # Score items based on similarity to user's rated items
    item_scores = {}
    for _, row in user_items.iterrows():
        item_id = row['item_id']
        if item_id in item_similarity_df.index:
            similar_items = item_similarity_df[item_id].sort_values(ascending=False)[1:51]
            
            for sim_item, similarity in similar_items.items():
                if sim_item not in user_items['item_id'].values:
                    if sim_item not in item_scores:
                        item_scores[sim_item] = 0
                    item_scores[sim_item] += similarity * row['rating']
    
    # Sort and return top N
    recommendations = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:n]
    return [item_id for item_id, score in recommendations]

# Generate predictions
item_based_predictions = {}
for user_id in test_users:
    item_based_predictions[user_id] = item_based_recommendations(user_id)

training_time_ib = time.time() - start_time
metrics_ib = calculate_metrics(item_based_predictions, test_data, movies)

print(f"✓ Training time: {training_time_ib:.2f} seconds")
print(f"✓ Precision@10:  {metrics_ib['Precision@10']:.4f}")
print(f"✓ Recall@10:     {metrics_ib['Recall@10']:.4f}")
print(f"✓ NDCG@10:       {metrics_ib['NDCG@10']:.4f}")
print(f"✓ Coverage:      {metrics_ib['Coverage']:.4f}")


✓ Training time: 52.75 seconds
✓ Precision@10:  0.2908
✓ Recall@10:     0.1871
✓ NDCG@10:       0.3514
✓ Coverage:      0.1153


### 7. SVD Matrix Factorization

In [30]:
start_time = time.time()

# Apply SVD
n_factors = 50
svd = TruncatedSVD(n_components=n_factors, random_state=42)
user_factors = svd.fit_transform(train_matrix)
item_factors = svd.components_.T

# Reconstruct rating matrix
predicted_ratings = np.dot(user_factors, item_factors.T)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings,
    index=train_matrix.index,
    columns=train_matrix.columns
)

def svd_recommendations(user_id, n=10):
    """Generate recommendations using SVD"""
    if user_id not in predicted_ratings_df.index:
        return []
    
    # Get user's predictions
    user_predictions = predicted_ratings_df.loc[user_id]
    
    # Remove already rated items
    user_rated = set(train_data[train_data['user_id'] == user_id]['item_id'].values)
    user_predictions = user_predictions[~user_predictions.index.isin(user_rated)]
    
    # Return top N
    recommendations = user_predictions.sort_values(ascending=False).head(n)
    return recommendations.index.tolist()

# Generate predictions
svd_predictions = {}
for user_id in test_users:
    svd_predictions[user_id] = svd_recommendations(user_id)

training_time_svd = time.time() - start_time
metrics_svd = calculate_metrics(svd_predictions, test_data, movies)

print(f"✓ Training time: {training_time_svd:.2f} seconds")
print(f"✓ Precision@10:  {metrics_svd['Precision@10']:.4f}")
print(f"✓ Recall@10:     {metrics_svd['Recall@10']:.4f}")
print(f"✓ NDCG@10:       {metrics_svd['NDCG@10']:.4f}")
print(f"✓ Coverage:      {metrics_svd['Coverage']:.4f}")
print(f"✓ Explained variance: {svd.explained_variance_ratio_.sum():.4f}")

✓ Training time: 0.64 seconds
✓ Precision@10:  0.3060
✓ Recall@10:     0.2078
✓ NDCG@10:       0.3694
✓ Coverage:      0.2705
✓ Explained variance: 0.4639


### 8. Hybrid (Item Based + SVD)

In [31]:
start_time = time.time()

def hybrid_recommendations(user_id, n=10, weight_ib=0.5, weight_svd=0.5):
    """Combine item-based and SVD recommendations"""
    ib_recs = item_based_recommendations(user_id, n=20)
    svd_recs = svd_recommendations(user_id, n=20)
    
    # Combine scores
    item_scores = {}
    for i, item_id in enumerate(ib_recs):
        item_scores[item_id] = weight_ib * (20 - i)
    
    for i, item_id in enumerate(svd_recs):
        if item_id in item_scores:
            item_scores[item_id] += weight_svd * (20 - i)
        else:
            item_scores[item_id] = weight_svd * (20 - i)
    
    # Sort and return top N
    recommendations = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)[:n]
    return [item_id for item_id, score in recommendations]

# Generate predictions
hybrid_predictions = {}
for user_id in test_users:
    hybrid_predictions[user_id] = hybrid_recommendations(user_id)

training_time_hybrid = time.time() - start_time
metrics_hybrid = calculate_metrics(hybrid_predictions, test_data, movies)

print(f"✓ Training time: {training_time_hybrid:.2f} seconds")
print(f"✓ Precision@10:  {metrics_hybrid['Precision@10']:.4f}")
print(f"✓ Recall@10:     {metrics_hybrid['Recall@10']:.4f}")
print(f"✓ NDCG@10:       {metrics_hybrid['NDCG@10']:.4f}")
print(f"✓ Coverage:      {metrics_hybrid['Coverage']:.4f}")

✓ Training time: 52.56 seconds
✓ Precision@10:  0.3284
✓ Recall@10:     0.2167
✓ NDCG@10:       0.3908
✓ Coverage:      0.2122


### 9. Model Comparision

In [32]:
# Create comparison dataframe with ALL metrics
comparison_df = pd.DataFrame({
    'Model': ['User-Based CF', 'Item-Based CF', 'SVD (MF)', 'Hybrid'],
    
    'Precision@10': [
        metrics_ub['Precision@10'],
        metrics_ib['Precision@10'],
        metrics_svd['Precision@10'],
        metrics_hybrid['Precision@10']
    ],
    
    'Recall@10': [
        metrics_ub['Recall@10'],
        metrics_ib['Recall@10'],
        metrics_svd['Recall@10'],
        metrics_hybrid['Recall@10']
    ],
    
    'NDCG@10': [
        metrics_ub['NDCG@10'],
        metrics_ib['NDCG@10'],
        metrics_svd['NDCG@10'],
        metrics_hybrid['NDCG@10']
    ],
    
    'Coverage': [
        metrics_ub['Coverage'],
        metrics_ib['Coverage'],
        metrics_svd['Coverage'],
        metrics_hybrid['Coverage']
    ],
    
    'Training Time (s)': [
        training_time_ub,
        training_time_ib,
        training_time_svd,
        training_time_hybrid
    ]
})

# formatting for cleaner output
print(comparison_df.to_string(index=False))

# Select best model (You can also change this to use 'NDCG@10' for better ranking quality)
best_model_metric = 'Precision@10'  # or 'NDCG@10'
best_model_idx = comparison_df[best_model_metric].idxmax()
best_model = comparison_df.iloc[best_model_idx]['Model']

print(f"\n🏆 BEST MODEL: {best_model}")
print(f"   {best_model_metric}: {comparison_df.iloc[best_model_idx][best_model_metric]:.4f}")
print(f"   Recall@10:    {comparison_df.iloc[best_model_idx]['Recall@10']:.4f}")
print(f"   NDCG@10:      {comparison_df.iloc[best_model_idx]['NDCG@10']:.4f}")

        Model  Precision@10  Recall@10  NDCG@10  Coverage  Training Time (s)
User-Based CF      0.320255   0.209530 0.383700  0.200357          72.284236
Item-Based CF      0.290774   0.187137 0.351412  0.115339          52.749058
     SVD (MF)      0.306045   0.207758 0.369437  0.270511           0.640822
       Hybrid      0.328420   0.216749 0.390792  0.212247          52.561404

🏆 BEST MODEL: Hybrid
   Precision@10: 0.3284
   Recall@10:    0.2167
   NDCG@10:      0.3908


### 10. Save Model

In [33]:
models = {
    'item_similarity': item_similarity_df,
    'svd_model': svd,
    'user_factors': user_factors,
    'item_factors': item_factors,
    'train_matrix': train_matrix,
    'movies': movies,
    'ratings': ratings
}

with open('recommendation_models.pkl', 'wb') as f:
    pickle.dump(models, f)

# Save comparison results
comparison_df.to_csv('model_comparison.csv', index=False)

print("✓ Saved recommendation_models.pkl")
print("✓ Saved model_comparison.csv")

print("\n" + "=" * 80)
print("TRAINING COMPLETE!")
print("=" * 80)
print(f"\nBest performing model: {best_model}")
print("Models are ready for deployment in Streamlit app!")

✓ Saved recommendation_models.pkl
✓ Saved model_comparison.csv

TRAINING COMPLETE!

Best performing model: Hybrid
Models are ready for deployment in Streamlit app!
